# Goal
Jacobo de la Cuesta-Zuluaga, May 2019.

Now that I have the gene clusters from `panX`, I can obtain a better annotation of them using `InterproScan` and `eggNOG mapper`.

# Init

In [3]:
import os
import pandas as pd
import subprocess
import json
from collections import Counter

/ebio/abt3_projects/software/miniconda3_gt4.4/envs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Var

In [4]:
# Work dir
work_dir = "/ebio/abt3_projects/vadinCA11/data/V11"
panX_dir = os.path.join(work_dir, "panx_output/pan-genome-analysis/data/Methanomassilii/vis/geneCluster")
annotation_dir = os.path.join(work_dir, "cluster_annot")
if not os.path.exists(annotation_dir):
    os.makedirs(annotation_dir)
# Misc
metacompass_env = "metacompass"
panx_env = "py2_panX"

# eggNOG database
eggnog_db = "/ebio/abt3_projects/databases_no-backup/Eggnog"

# interpro executable_dir
interpro_bin = "/ebio/abt3_projects/vadinCA11/bin/interpro/interproscan_5"
interpro_sh = os.path.join(interpro_bin, "interproscan.sh")

# ALPs
ALP_dir = os.path.join(work_dir, "ALPs/Mbrevi_alp/unaligned")

# PanX gene cluster table
Gene_clust = pd.read_csv("/ebio/abt3_projects/vadinCA11/data/V11/panx_output/tables/gene_clust_summary.txt", 
                         sep='\t', header=0)

# Annotate gene clusters

## Consensus sequence of gene clusters

For `eggNOG` and `Interpro` I will use the consensus sequence of each gene cluster. I will first create a fasta file with the consensus of all clusters

In [112]:
# Create list of name of resulting clusters from PanX table
# and add name of aa alignment file
cluster_names = Gene_clust["MSA"].tolist()
cluster_files = []
for name in cluster_names:
    file = os.path.join(panX_dir, name + "_aa_aln_reduced.fa.gz")
    cluster_files.append(file)

In [120]:
# panX_output = os.listdir(panX_dir)
# cluster_aligns = [fasta for fasta in panX_output if "_aa_aln_reduced.fa.gz" in fasta]
cluster_files = cluster_files

In [ ]:
# Create a single file with all consensus sequences
# File path
consensus_gapped_file = os.path.join(annotation_dir, "consensus_gapped.fasta")
!touch $consensus_gapped_file
for fasta in cluster_aligns:
    fasta_path = os.path.join(panX_dir, fasta)
    # Change header of sequence to cluster name
    cluster_name = fasta.replace("_aa_aln_reduced.fa.gz", "")
    header = ">{cluster}".format(cluster = cluster_name)
    # Extract first sequence from cluster align file
    zcat_cmd = 'zcat {file} | awk "/^>/ {{n++}} n>1 {{exit}} {{print}}"'.format(file = fasta_path)
    # Replace header
    sed_cmd = "{zcat} | sed 's/^>.*$/{header}/' >> {out_file}".format(zcat = zcat_cmd, 
                                                                  header = header, 
                                                                  out_file =  consensus_gapped_file)

    !$sed_cmd


gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe



In [70]:
# Ungap consensus sequences and format
consensus_file = os.path.join(annotation_dir, "consensus.fasta")
ungap_cmd = "cat {in_fasta} | seqtk seq -A | tr -d -  > {out_fasta}".format(in_fasta = consensus_gapped_file, 
                                                                           out_fasta = consensus_file)

ungap_job = 'bash -c "source activate {0}; {1}"'.format(metacompass_env, ungap_cmd)
print(ungap_job)
#!$ungap_job

bash -c "source activate metacompass; cat /ebio/abt3_projects/vadinCA11/data/V11/cluster_annot/consensus_gapped.fasta | seqtk seq -A | tr -d -  > /ebio/abt3_projects/vadinCA11/data/V11/cluster_annot/consensus.fasta"


## Execute InterproScan

In [71]:
interpro_cmd = "{interpro} -i {in_fasta} \
    --output-dir {out_dir} \
    --cpu {cpu} \
    --tempdir {tmp} \
    --formats tsv"
interpro_cmd = interpro_cmd.format(interpro = interpro_sh, 
                                   in_fasta = consensus_file, 
                                   out_dir = annotation_dir, 
                                   tmp =" /tmp/global2/jdelacuesta",
                                   cpu = 20)

interpro_job = 'bash -c "source activate {0}; {1}"'.format("metacompass", interpro_cmd)
print(interpro_job)
#!$interpro_job

bash -c "source activate metacompass; /ebio/abt3_projects/vadinCA11/bin/interpro/interproscan_5/interproscan.sh -i /ebio/abt3_projects/vadinCA11/data/V11/cluster_annot/consensus.fasta     --output-dir /ebio/abt3_projects/vadinCA11/data/V11/cluster_annot     --cpu 20     --tempdir  /tmp/global2/jdelacuesta     --formats tsv"


## Execute eggNOG mapper

In [122]:
eggnog_cluster = """#!/bin/bash
#$ -N {name}
#$ -pe parallel {cpu}
#$ -l h_vmem=10G
#$ -l h_rt=200:0:0
#$ -o /ebio/abt3_projects/vadinCA11/data/V11/eggnog/cluster_jobs/logs
#$ -j y
#$ -wd {workdir}
#$ -m ea
#$ -M jdelacuesta@tuebingen.mpg.de

export PATH='/ebio/abt3_projects/software/miniconda3_gt4.4/envs/eggNOG/bin':$PATH

emapper.py -i {input_file} \
    --database {DB_type} \
    --data_dir {eggnog_database}  \
    --target_orthologs all \
    --output_dir {output_dir} \
    --output {base_name} \
    --scratch_dir {tmp_folder} \
    --temp_dir {tmp_folder} \
    --go_evidence non-electronic \
    --cpu {cpu} \
    --usemem \
    --override \
    -m {method}
"""

In [128]:
job_name = "eggnog_clusters.sh"

eggnog_output = os.path.join(annotation_dir, "eggNOG")
script_eggnog = os.path.join(eggnog_output, job_name)


if not os.path.exists(eggnog_output):
    os.makedirs(eggnog_output)

prot_eggnog = eggnog_cluster.format(name = job_name, 
                                    cpu = 20, 
                                    input_file = consensus_file, 
                                    DB_type = "arch", 
                                    workdir = eggnog_output, 
                                    eggnog_database = eggnog_db, 
                                    output_dir = eggnog_output, 
                                    base_name = "Clusters", 
                                    tmp_folder = "/tmp/global/jdelacuesta/", 
                                    method = "diamond")

In [129]:
fh = open(script_eggnog, "w")
fh.write(prot_eggnog)
fh.close()
!qsub $script_eggnog

Your job 4633871 ("eggnog_clusters.sh") has been submitted


# Annotate *M. smithii* ALPs

## Consensus sequence of *M. smithii* ALPs

In [96]:
ALP_files = os.listdir(ALP_dir)
ALP_seqs = [fasta for fasta in ALP_files if "faa" in fasta]

In [101]:
# Create a single file with all consensus sequences
# File path
ALPs_file = os.path.join(annotation_dir, "Msmithii_ALPs.fasta")
!touch $ALPs_file
for fasta in ALP_seqs:
    fasta_path = os.path.join(ALP_dir, fasta)
    # Change header of sequence to cluster name
    cluster_name = fasta.replace(".faa", "")
    header = ">{cluster}".format(cluster = cluster_name)
    # Extract first sequence from cluster align file
    cat_cmd = 'cat {file} | awk "/^>/ {{n++}} n>1 {{exit}} {{print}}"'.format(file = fasta_path)
    # Replace header
    sed_cmd = "{cat} | sed 's/^>.*$/{header}/' >> {out_file}".format(cat = cat_cmd, 
                                                                  header = header, 
                                                                  out_file =  ALPs_file)
#     print(sed_cmd)
    !$sed_cmd

cat: write error: Broken pipe


## Execute InterproScan

In [102]:
interpro_cmd = "{interpro} -i {in_fasta} \
    --output-dir {out_dir} \
    --cpu {cpu} \
    --tempdir {tmp} \
    --formats tsv"
interpro_cmd = interpro_cmd.format(interpro = interpro_sh, 
                                   in_fasta = ALPs_file, 
                                   out_dir = annotation_dir, 
                                   tmp =" /tmp/global2/jdelacuesta",
                                   cpu = 20)

interpro_job = 'bash -c "source activate {0}; {1}"'.format("metacompass", interpro_cmd)
print(interpro_job)
#!$interpro_job

bash -c "source activate metacompass; /ebio/abt3_projects/vadinCA11/bin/interpro/interproscan_5/interproscan.sh -i /ebio/abt3_projects/vadinCA11/data/V11/cluster_annot/Msmithii_ALPs.fasta     --output-dir /ebio/abt3_projects/vadinCA11/data/V11/cluster_annot     --cpu 20     --tempdir  /tmp/global2/jdelacuesta     --formats tsv"
